In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'country_demo', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change
Country_demo = Base.classes.country_demo

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())
results_temp = session.query(Temp_change)
temp_df = pd.read_sql(results_temp.statement, session.connection())

results_Demo = session.query(Country_demo)
demo_df = pd.read_sql(results_Demo.statement, session.connection(),index_col='index')

# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]

In [7]:

country='China, Hong Kong SAR'

#Get Average Temp Change per year for the selected Country
temp_country = meteor_df.loc[meteor_df['Area'] == country].drop(['field1','Area','Months', 'Element', 'Unit'], 1).reset_index(drop=True).T.reset_index()
temp_country= temp_country.rename(columns = {'index':'Year', 0:'Avg Temp'})

#check for dtypes before merging
temp_country.dtypes
temp_country

,Year,Avg Temp
0,1961,0.047
1,1962,-0.090
2,1963,0.289
3,1964,0.079
4,1965,0.115
5,1966,0.616
6,1967,0.032
7,1968,-0.662
8,1969,-0.017
9,1970,-0.414


In [8]:
#parse df object to datetime object
from datetime import datetime
temp_country['Year'] = pd.to_datetime(temp_country['Year'],format="%Y")
temp_country['Year'] = pd.DatetimeIndex(temp_country['Year']).year
temp_country.dtypes
temp_country

,Year,Avg Temp
0,1961,0.047
1,1962,-0.090
2,1963,0.289
3,1964,0.079
4,1965,0.115
5,1966,0.616
6,1967,0.032
7,1968,-0.662
8,1969,-0.017
9,1970,-0.414


In [9]:
co2_country = emission_df.loc[emission_df['Entity']==country].drop(['Entity', 'Code'], 1).reset_index(drop =True)
co2_country= co2_country.rename(columns = {'Year':'Year', 'AnnualCO2emissions':'CO2 Emission'})

co2_country.dtypes
co2_country

,Year,CO2 Emission


In [10]:
#Merge Temp_change Co2 Emission df
merged_Temp_co2 =temp_country.merge(co2_country, on="Year", left_index=False, right_index=False,)
merged_Temp_co2

#convert Co2 Emission to Mega Ton
merged_Temp_co2['CO2 Emission'] = round(merged_Temp_co2['CO2 Emission'].apply(lambda x: x/1000000),3)
merged_Temp_co2

,Year,Avg Temp,CO2 Emission


In [11]:
Scatter_obj = {
                "Country":country,
                "Year":list(np.ravel(merged_Temp_co2['Year'])),
                "Avg Temp Change":list(np.ravel(merged_Temp_co2['Avg Temp'])), 
                "Co2 Emission":list(np.ravel(merged_Temp_co2['CO2 Emission']))
            }

Scatter_obj

{'Country': 'China, Hong Kong SAR',
 'Year': [],
 'Avg Temp Change': [],
 'Co2 Emission': []}

In [12]:
#Get Average Temp Change per year for the selected Country , Transpose table and reset index
country = 'United States of America'
temp_country = meteor_df.loc[meteor_df['Area'] == country].drop(['field1','Area','Months', 'Element', 'Unit'], 1).reset_index(drop=True).T.reset_index()
temp_country= temp_country.rename(columns = {'index':'Year', 0:'Avg Temp'})
#check for dtypes before merging
temp_country.dtypes
#parse df object to datetime object and get only Years
temp_country['Year'] = pd.to_datetime(temp_country['Year'],format="%Y")
temp_country['Year'] = pd.DatetimeIndex(temp_country['Year']).year
#get Co2 Emission Data , drop unwated fields and reset index
co2_country = emission_df.loc[emission_df['Entity']==country].drop(['Entity', 'Code'], 1).reset_index(drop =True)
co2_country= co2_country.rename(columns = {'Year':'Year', 'AnnualCO2emissions':'CO2 Emission'})
#check for data types before merging
co2_country.dtypes
#Merge Temp_change Co2 Emission df
merged_Temp_co2 =temp_country.merge(co2_country, on="Year", left_index=False, right_index=False)
#convert Co2 Emission to Mega Ton
merged_Temp_co2['CO2 Emission'] = round(merged_Temp_co2['CO2 Emission'].apply(lambda x: x/1000000),3)
# Get data in a object
Scatter_obj1 = {
            "Country":country,
            "Year":list(np.ravel(merged_Temp_co2['Year'])),
            "Avg Temp Change":list(np.ravel(merged_Temp_co2['Avg Temp'])), 
            "Co2 Emission":list(np.ravel(merged_Temp_co2['CO2 Emission']))
        }
Scatter_obj1

{'Country': 'United States of America',
 'Year': [1961,
  1962,
  1963,
  1964,
  1965,
  1966,
  1967,
  1968,
  1969,
  1970,
  1971,
  1972,
  1973,
  1974,
  1975,
  1976,
  1977,
  1978,
  1979,
  1980,
  1981,
  1982,
  1983,
  1984,
  1985,
  1986,
  1987,
  1988,
  1989,
  1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019],
 'Avg Temp Change': [0.0,
  -0.023,
  0.418,
  -0.285,
  -0.358,
  -0.213,
  0.073,
  -0.082,
  -0.274,
  -0.031,
  -0.416,
  -0.249,
  -0.018,
  0.08,
  -0.387,
  -0.124,
  0.4,
  0.046,
  -0.297,
  0.413,
  0.867,
  -0.337,
  0.537,
  -0.226,
  -0.02,
  0.584,
  0.91,
  0.469,
  0.325,
  0.672,
  0.525,
  0.395,
  0.005,
  0.461,
  0.626,
  -0.011,
  0.385,
  1.261,
  0.747,
  0.991,
  0.776,
  0.946,
  0.986,
  0.855,
  1.143,
  1.013,
  1.134,
  0.203,
  0.386,
  0.679,
